# Part 3: Peer-to-peer Message Behaviour Data Analysis

---

### Install Python packages (pip only)

In [1]:
#e.g., %pip install some-package
%pip install networkx numpy scipy pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Import Python packages

In [1]:
#e.g., import some-package
import pandas as pd
import networkx as nx
import numpy as np

---

### Task 1 of 2

Examine the file "p2p_msg.csv" which represents messaging behaviour between users on a messaging platform. Each row has four columns, representing a single event where a person (person_a) messaged another person (person_b) on some date (date) at some time of day (time). From this, answer the following questions:

##### Q1. Build a suitable network to represent social connections based on the messaging behaviour that took place in the first 28 days. In doing so, assume that one or more messages from one person to another represents a mutual underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point). 

In [2]:
#CODE:
# using pandas to read the csv and modify it 
df = pd.read_csv("p2p_msg.csv")

# converting objects date and time to datetime
df['Datetime'] = pd.to_datetime(df.date +' '+df.time, dayfirst=True)

# getting the first date on the dataframe 
first_date = df['Datetime'].min()

# making the last date to be 28 days after the first date
last_date = first_date + pd.Timedelta(days = 28)

relevant_df = df[df['Datetime'] <= last_date]

social_graph = nx.Graph()
# adding the people as nodes
nodes = list(set(relevant_df["person_a"].tolist() + relevant_df['person_b'].tolist()))
social_graph.add_nodes_from(nodes)

# adding edges based on person_a and person_b connections

for index, data  in relevant_df.iterrows(): 
    person_a = data['person_a']
    person_b = data['person_b']
    social_graph.add_edge(person_a, person_b)

print("Social network built")
print(f"Number of nodes: {social_graph.number_of_nodes()}")
print(f"Number of edges: {social_graph.number_of_edges()}")

degree_centrality = nx.degree_centrality(social_graph)

most_connected_person = max(degree_centrality, key=degree_centrality.get)

print(f"Most connected person: {most_connected_person}")

Social network built
Number of nodes: 1147
Number of edges: 6516
Most connected person: 400


##### Q2. Using the largest connected component of the network constructed in Task 1, Q1. What is the mean, median and the standard deviation of the differences between the maximum degree of separation of each individual and the average distance between the individual and all others?

In [3]:
#CODE:
# getting the largest connected component
largest_connected_component = max(nx.connected_components(social_graph), key=len)

# calculating shortest paths from each node
all_paths = dict(nx.all_pairs_shortest_path(social_graph.subgraph(largest_connected_component)))

max_degree_of_separation = nx.diameter(social_graph.subgraph(largest_connected_component))
differences = [max_degree_of_separation - all_paths[node][other][-1] for node in largest_connected_component for other in largest_connected_component if node != other]

# Calculate mean, median and standard deviation
mean_difference = sum(differences) / len(differences)
median_difference = sorted(differences)[len(differences) // 2]
std_difference = pd.Series(differences).std()

# Print the results
print("Mean difference:", mean_difference)
print("Median difference:", median_difference)
print("Standard deviation:", std_difference)

Mean difference: -578.753711790393
Median difference: -578
Standard deviation: 334.5213413670821


##### Q3. Build another suitable network to represent social connections based on ALL message behaviour in the dataset. In doing so, assume that one or messages from one person to another represents a MUTUAL underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point).Can the social phenomenon, ‘Triadic Closure’, be supported for the common nodes that exist in both the network created from behaviour for the first 28 days (i.e., from Task 1, Q1) and the network built from all message behaviour?

In [4]:
#CODE:

# using df from the task 1
full_graph = nx.Graph()
# adding the people as nodes
nodes = list(set(df["person_a"].tolist() + df['person_b'].tolist()))
full_graph.add_nodes_from(nodes)

# adding edges based on person_a and person_b connections

for index, data  in df.iterrows(): 
    person_a = data['person_a']
    person_b = data['person_b']
    full_graph.add_edge(person_a, person_b)

    
common_nodes = set(social_graph.nodes())&set(full_graph.nodes())

subgraph_social_graph = social_graph.subgraph(common_nodes)



# using function from part 2
def triad_occurences(Graph, triad_type): 
    """
    Function to count triads in the largest strongly connected networks
    
    """
    count = 0 
    
    for node in Graph.nodes(): 
        
        neigbors = list(Graph.neighbors(node))
        # getting the two neighbors for comparison
        for i in range(len(neigbors) - 1): 
            for j in range(i+1, len(neigbors)): 
                neighbor1, neighbor2 = neigbors[i], neigbors[j]
                
                if triad_type == "mutual" and Graph.has_edge(neighbor1, neighbor2) and Graph.has_edge(neighbor2, neighbor1):
                    count += 1
                elif triad_type == "mixed" and (Graph.has_edge(neighbor1, neighbor2) or Graph.has_edge(neighbor2, neighbor1)):
                    count += 1
    
    return count


triad_28 = triad_occurences(subgraph_social_graph, "mutual")

clustering_coefficients = nx.clustering(subgraph_social_graph)
# getting average clustering coefficient in 28 days subgraph
average_clustering_coefficient = sum(clustering_coefficients.values()) / len(clustering_coefficients)

print(f"Average Clustering Coefficient for the common nodes (first 28 days): {average_clustering_coefficient:.2f}")
print(f" Mutual social connections: {triad_28}")

Average Clustering Coefficient for the common nodes (first 28 days): 0.11
 Mutual social connections: 13968


##### Q4. What hypothetical, non-existent edges would need to be added to the network representing all message behaviour (i.e., from Task 1, Q3) such that a message could pass along a path from any node to any other? In doing so, aim to minimise the number of edges that would be needed as well as the longest shortest path in the network as a result.

In [5]:
#CODE:
# using full_graph from Q3
# Find connected components in the network
connected_components = list(nx.connected_components(full_graph))

# Sort the connected components by size in descending order
sorted_components = sorted(connected_components, key=len, reverse=True)

# List to store the edges that should be added to minimize the number of edges and the longest shortest path
edges_to_add = []

# connect the components by adding an edge from the last node in the largest component
# to the first node in the next largest component

for i in range(len(sorted_components) - 1):
    # Get a node from the largest component (last node for variety)
    node_from_largest_component = next(iter(sorted_components[i + 1]))
    
    # Get a node from the next component (first node for variety)
    node_from_next_component = next(iter(sorted_components[i]))
    
    # adding this edge to the list
    edges_to_add.append((node_from_largest_component, node_from_next_component))

# Now we have a list of edges that, if added, would connect all components
# Let's print them out
print(f"The following edges need to be added to minimize the number of edges and the longest shortest path:")

print(edges_to_add)
for edge in edges_to_add:
    print(edge)
# Optionally, you can add these edges to the network and calculate the new longest shortest path
for edge in edges_to_add:
    full_graph.add_edge(*edge)

# Calculate the diameter of the network which is the longest shortest path
network_diameter = nx.diameter(full_graph)
print(f"The longest shortest path in the network (network diameter) is now: {network_diameter}")

The following edges need to be added to minimize the number of edges and the longest shortest path:
[(10, 1), (229, 10), (1192, 229), (1797, 1192), (1812, 1797)]
(10, 1)
(229, 10)
(1192, 229)
(1797, 1192)
(1812, 1797)
The longest shortest path in the network (network diameter) is now: 11


### Task 2 of 2 

Using the largest connected component of the network constructed from all data in Task 1, Q2, assume the role of an outsider with complete visibility of the network that now wishes to spread a hypothetical message such that everyone in the component would know the information it contained as quickly as possible. Assume that messages will now spread in sequential timesteps using the following mechanism. If an individual is told the message at timestep 𝑡, the individual will forward the message to all of their direct connections at timestep 𝑡+1. Individuals can therefore be told the message more than once. From this, answer the following questions:

##### Q1. If you could only select 1 individual to tell at timestep 0, what set of nodes could you select from which would result in the message being received by everyone in the fewest timesteps as possible and what would the number of timesteps be?

In [6]:
#CODE:
# using the largest connectecd component generated above
largest_connected_component = max(nx.connected_components(full_graph), key=len)
G = full_graph.subgraph(largest_connected_component)


#getting the eccentricity
eccentricities = nx.eccentricity(G)
# finding nodew with smallest eccentricity
minimum_eccen = min(eccentricities.values())
centers = [node for node, ecc in eccentricities.items() if ecc == minimum_eccen]



print(f"Nodes that I would select that would result to message being received by everyone in the fewest timesteps({minimum_eccen})")
print(centers)

Nodes that I would select that would result to message being received by everyone in the fewest timesteps(6)
[1, 10]


##### Q2. If you had to select any 5 individuals to tell at timestep 0, can the message be received by everyone in fewer timesteps than the single individual selection in Q1? In determining your answer, use one or more appropriate network connectivity measures, rather than an exhaustive search through every combination of nodes in the network.

In [7]:
#CODE:
# getting the betweeness centrality 
b_centrality = nx.betweenness_centrality(G) 

d_centrality = nx.degree_centrality(G)

# Sorting nodes by centrality (descending order)
sorted_by_degree = sorted(d_centrality.items(), key=lambda item: item[1], reverse=True)
sorted_by_betweenness = sorted(b_centrality.items(), key=lambda item: item[1], reverse=True)

# Select top nodes with high degree centrality (2-3 nodes)
high_degree_nodes = [node for node, _ in sorted_by_degree[:5]]

# Select top nodes with high betweenness centrality (1-2 nodes)
high_betweenness_nodes = [node for node, _ in sorted_by_betweenness[:2]]

# Combine and choose a diverse set of 5 nodes (adjust selection logic as needed)
potential_seed_nodes = list(set(high_degree_nodes) | set(high_betweenness_nodes))
chosen_seed_nodes = potential_seed_nodes[:5]  # Select top 5 from the combined list

print(f"Choosen nodes: {chosen_seed_nodes}")


Choosen nodes: [32, 105, 400, 103, 9]
